<a href="https://colab.research.google.com/github/emerdg/Narrador-Interativo---Alura-Gemini/blob/master/Narrador_Interativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Narrador Interativo com Gemini 🚀

Bem-vindo à sua aventura narrativa! Siga os passos abaixo para configurar e começar a jogar.

## Primeiro Passo: Insira sua API Key

Clique no ícone de chave (🔑) no menu lateral esquerdo para abrir o painel de `Secrets`. Adicione uma nova chave com o nome `GOOGLE_API_KEY` e cole sua chave de API do Google Gemini (obtida em [Google AI Studio](https://aistudio.google.com/app/apikey)).

**⚠️ Sua API KEY é confidencial e só será usada por este notebook!**

## Segundo Passo: Preparação do Ambiente

Rode o código abaixo para instalar as bibliotecas necessárias e configurar seu acesso ao Gemini. O Colab cuidará de tudo nos bastidores.

In [ ]:
import os
import time
import sys

# Importar display e Markdown para formatação visual
from IPython.display import display, Markdown, HTML # Adicionado HTML
import textwrap # Já importado, útil para formatar texto

################################################################################
# Função para efeito de digitação (útil para mensagens de status na configuração)
def digitar(texto):
    for letra in texto:
        sys.stdout.write(letra)
        sys.stdout.flush()
        time.sleep(0.01)              # velocidade de digitação
    print()                           # Adiciona uma nova linha ao final
################################################################################

# Função para exibir texto do Narrador com formatação visual (caixa)
def display_narrator_response(text):
    # Formatar o texto usando HTML e Markdown para criar uma caixa com estilo
    html_content = f'''
    <div style=\"border: 1px solid #777; padding: 15px; margin: 15px 0; border-radius: 8px; background-color: #333; color: #fff;\">
        🗣️ **NARRADOR:**<br>
        {textwrap.fill(text.strip(), width=60)} <!-- Quebra de linha para melhor visualização -->
    </div>
    '''
    display(HTML(html_content))

# Função para exibir títulos de seção formatados
def display_section_heading(title):
    display(Markdown(f"\n## {title}\n---")) # Título grande com linha separadora


display_section_heading("Preparando o Ambiente...")

digitar(texto='Pera ai, deixa eu pegar umas coisas aqui...')
!pip install -q google-genai          # Instalando Bibliotéca do Gemini

digitar(texto=
        '''\"Oi, google? Você deixa eu usar essas coisas aqui?
        \n Rapidinho, coisa boba... \"
        \n
        '''
        )
!pip install -q google-adk            # Instalando Bibliotéca do ADK (Agent Development Kit)

# Inserindo essas frases bobas a cada etapa eu consigo saber em que etapa o
# codigo se encontra ;)

from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY') # Copiando sua API KEY para a memória;


################################################################################
################################### Shor de Importação! Carrega tudo aqui, bora!

digitar(texto='Chegou! Deixa eu colocar cada coisa em seu lugar, só um segundo...')

from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
# import textwrap # Removido, já importado acima
# from IPython.display import display, Markdown # Removido, já importado acima
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")
################################################################################


display_section_heading("Pronto para Configurar!")
digitar(texto=
    '''Tudo certo!
    \n Agora o Google nos emprestou o Gemini por um tempo. Vamos para o próximo passo!
    \n Você já pode executar o próximo bloco para configurar seu jogo.
    '''
    )




In [ ]:
modelo_grande = "gemini-2.0-flash"
modelo_rapido = "gemini-2.0-flash-lite"

# Gerenciador de agentes - Mantido como estava
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

################################################################################ AGENTE CONFIGURACAO DO NARRADOR
def agente_conf_narrador(descricao, referencias):
  conf_narrador = Agent(
      name="agente_conf_narrador",
      model=modelo_rapido,
      description="Agente responsável por criar o nosso narrador.",
      tools=[google_search],
      instruction='''
      Você é um agente responsável por configurar um narrador, definir qual é seu humor, gostos e estilo de narrativa baseado nas intruções do leitor.
      você deve se basear nas seguintes informações:
      Use DESCRICAO para caracterizar e descrever o narrador.
      Use REFERENCIAS e usando goole (google_search) para recuperar informações sobre os gostos e interesses do narrador.
      Foque no que é mais importante para a narração do jogo.
      CASO o leitor não dê um nome, crie um nome para ele ou ela.
      Se faltar informação, crie algo bem divertido para o leitor.

      IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento.
      IMPORTANTE: De forma alguma o narrador deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou violento.
      '''
  )
  entrada_do_agente_conf_narrador = f'''
  DESCRICAO: {descricao} \n
  REFERENCIAS: {referencias}
  '''
  res_conf_narrador = call_agent(conf_narrador, entrada_do_agente_conf_narrador)
  return res_conf_narrador

  ################################################################################ AGENTE CONFIGURACAO DA HISTORIA
def agente_conf_historia(onde, referencias, estilo, comportamento):
  conf_historia = Agent(
      name="agente_conf_historia",
      model=modelo_grande,
      description="Agente responsável por criar o cenário e a história.",
      tools=[google_search],
      instruction='''
      Você é um agente responsável por configurar a história que nosso narrador irá contar, definir aonde ela irá se passar, que gênero narrativo ela deve seguir e qual o foco da narrativa.
      você deve se basear nas seguintes informações:
      Use LUGAR para saber em que lugar se passa essa história.
      Use REFERENCIAS e usando goole (google_search) para recuperar informações que te ajudem a construir este cenario da melhor maneira possível
      Use ESTILO para adaptar a história a um estilo literário coerente e usando goole (google_search) encontre bons exemplos.
      Use COMPORTAMENTO para entender como essa hsitória será contada, seja como um narrador de um RPG de mesa, um livro jogo com alternativas ou de outra forma solicitada pelo leitor.
      Crie lugares e personágens importantes que podem ou não ser apresentados na história.
      Foque no que é mais importante para a narração da história, dando a melhor atmosfera possível para o ambiente e vida ao jogo.
      Se faltar informação, crie algo bem divertido para o leitor.

      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizados, vulgarizados ou apresentar violência além do necessário para uma boa narrativa.
      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizados, vulgarizados ou apresentar violência além do necessário para uma boa narrativa
      '''
  )
  entrada_do_agente_conf_historia = f'''
  LUGAR: {onde}
  REFERENCIAS: {referencias}
  ESTILO: {estilo}
  COMPORTAMENTO: {comportamento}
  '''
  res_conf_historia = call_agent(conf_historia, entrada_do_agente_conf_historia)
  return res_conf_historia

################################################################################ AGENTE CRIACAO DO NARRADOR
# Este agente foi removido, pois sua função foi integrada ao system_instruction do chat principal
# mantendo apenas os agentes de configuração para gerar as descrições iniciais.
# def agente_narrador(res_conf_narrador, res_conf_historia, res_conf_personagem):
#   agente_narrador = Agent(
#       name="agente_narrador",
#         model=modelo_rapido,
#       description="Agente responsável por contar a história.",
#       instruction='''
#       Você é um Narrador de histórias.
#       ... (instruções)
#       '''
#       )
#   entrada_do_agente_narrador = f'''
#   ... (entradas)
#   '''
#   res_agente_narrador = call_agent(agente_narrador, entrada_do_agente_narrador)
#   return res_agente_narrador

################################################################################ AGENTE CONFIGURACAO DO PERSONAGEM
def agente_conf_personagem(Persona, referencias, relacoes):
  conf_personagem = Agent(
      name="agente_conf_personagem",
      model=modelo_grande,
      description="Agente responsável por criar o personagem jogador.",
      tools=[google_search],
      instruction='''
      Você é um agente responsável por configurar o personágem do jogador, podendo ser mais de um, de acordo com a solicitação do leitor.
      Você deve se basear nas seguintes informações:
      Use PERSONA para descrever o personágem, gostos e aparencia.
      Use REFERENCIAS e usando goole (google_search) para recuperar informações que te ajudem a construir este personágem da melhor maneira possível.
      Use RELACOES para localizar o personagem na história e estabelecer ligações dele com lugares e personagens importantes.
      Inclua alguns pertences relevantes, gostos pessoais, inimisades e relações com outros personagens da narrativa.
      Foque no que é mais importante para a narração da história, tornando o personagem parte vital do jogo, sendo um ou mais personagens.
      Se faltar informação, crie algo bem divertido para o leitor.

      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizados, vulgarizados ou apresentar violência além do necessário para uma boa narrativa.
      IMPORTANTE: De forma alguma o ambiente ou os personagens devem ser sexualizado, vulgarizados ou apresentar violência além do necessário para uma boa narrativa
      '''
  )

  entrada_do_agente_conf_personagem = f'''
  PERSONA: {Persona}
  REFERENCIAS: {referencias}
  RELACOES: {relacoes}
  '''
  res_conf_personagem = call_agent(conf_personagem, entrada_do_agente_conf_personagem)
  return res_conf_personagem

display_section_heading("Configuração da História e Personagem")
digitar(texto=
    '''Agora vamos definir os detalhes do seu jogo! Responda às perguntas abaixo para moldar sua aventura.
    '''
    )

In [ ]:
display_section_heading("Detalhes do Narrador")
descricao = input("\n✨ Por favor, descreva quem será o narrador que irá contar a sua história. \n Inclua informações como humor, gostos e estilos. \n Isso ajudará a dar mais naturalidade e personalidade a forma como a história será contada. \n")
referencias_narrador = input("\n📚 O que ele gosta de ler, assistir ou jogar? \n Não necessariamente isso será incluso na história, mas ajuda a dar personalidade ao narrador. \n")

# CRIANDO NARRADOR ################################################
if not descricao:
  descricao = "Um narrador curioso e que adora mistérios."

if not referencias_narrador:
  referencias_narrador = "Livros de Agatha Christie, jogos de detetive e filmes clássicos de suspense."

digitar("\n Ok, só um minuto, estou montando o narrador... \n Encaixando algumas peças, lendo o manual...")

final_narrador = agente_conf_narrador(descricao, referencias_narrador)

# Exibir o resultado da configuração do narrador
display(Markdown("### 👤 Seu Narrador está pronto!"))
display(Markdown(final_narrador))


display_section_heading("Detalhes da História")
onde = input("\n🗺️ Onde se passará nossa história? \n Pode ser um lugar no mundo real ou na fantasia. \n")
referencias_historia = input("\n🎬 Quer dar uma referencia de alguns mundos legais que você gosta? \n")
estilo =  input("\n✍️ Em que estilo de escrita devemos nos basear? \n literatura de horror, romance, young-adult, ficção científica? \n")
comportamento =  input("\n🎮 Como será nosso jogo? Um RPG, onde o narrador conta a história e você escreve ou escolhe a continuação? Terá conflitos como combates ou charadas? \n")

# CRIANDO HISTORIA ################################################
if not onde:
  onde = "uma cidade costeira misteriosa"

if not referencias_historia:
  referencias_historia = "Os contos de H.P. Lovecraft e a série Stranger Things"

if not estilo:
  estilo = "mistério e horror cósmico"

if not comportamento:
  comportamento = "Um jogo de investigação e escolhas, focado em resolver mistérios. O narrador apresentará dilemas e o jogador escolherá o caminho."

digitar("\n Tá certo, minha equipe de roteiristas está escrevendo nossa história. \n Só mais um segundo!")

final_historia = agente_conf_historia(onde, referencias_historia, estilo, comportamento)

# Exibir o resultado da configuração da história
display(Markdown("### 🌎 O Palco da sua História está montado!"))
display(Markdown(final_historia))


display_section_heading("Detalhes do Personagem")
Persona = input("\n🦸‍♀️ Quem é o seu personagem neste mundo? \n Dê uma breve descrição dele (aparência, profissão, personalidade). \n")
referencias_personagem = input("\n🎭 Quem, no mundo da ficção, se parece com o seu personágem? \n")

# CRIANDO PERSONAGEM ################################################
if not Persona:
  Persona = "Um jovem curioso, recém-chegado na cidade, em busca de respostas sobre um parente desaparecido."

if not referencias_personagem:
  referencias_personagem = "Indiana Jones jovem e detetives clássicos"

digitar("\n Legal, agora nossa equipe de maquiagem e figurino vai trabalhar! \n Mais um segundo, por favor...")

# Passamos a configuração da história como referência para o personagem
final_personagem = agente_conf_personagem(Persona, referencias_personagem, final_historia)

# Exibir o resultado da configuração do personagem
display(Markdown("### 🚶‍♂️ Seu Personagem está pronto!"))
display(Markdown(final_personagem))


################################################################################ INICIO DA NARRATIVA

display_section_heading("Início da Aventura!")

display(Markdown("\n✨ **OK! Vamos começar a contar sua história!** ✨"))
display(Markdown("\n*Digite `!FIM` a qualquer momento para sair do jogo.*\n"))


client = genai.Client()

# System Instruction para o CHAT principal (o narrador)
# Inclui as descrições geradas pelos agentes de configuração
chat_config = types.GenerateContentConfig(
    system_instruction = f'''
    Você é um narrador de histórias interativas. Sua função é descrever as cenas, apresentar o personagem do jogador no mundo e guiar a narrativa com base nas escolhas do jogador.
    Mantenha a persona, o estilo e o comportamento definidos. Você é a voz e os olhos do mundo para o jogador.

    Aqui estão os detalhes que você deve seguir:
    **Sua Persona (o Narrador):** {final_narrador}
    **A História e o Mundo:** {final_historia}
    **O Personagem do Jogador:** {final_personagem}

    **Instruções para Narrar:**
    1. Seja criativo e envolvente nas descrições.
    2. Descreva as cenas com cuidado, focando na atmosfera e nos detalhes relevantes para a história.
    3. Mantenha os parágrafos concisos, no máximo 6 parágrafos por resposta para não sobrecarregar o jogador.
    4. Apresente claramente as opções ou o tipo de ação que o jogador pode tomar, de acordo com o COMPORTAMENTO definido para o jogo.
    5. Reaja às escolhas do jogador, avançando a narrativa de forma coerente com o mundo, história e personagem.

    **Restrições Importantes:**
    *   De forma alguma o conteúdo gerado deve ser sexualizado, menor de idade (em aparencia ou idade), vulgar ou excessivamente violento (além do necessário para o gênero ou atmosfera definida).
    *   Siga as diretrizes de segurança do conteúdo do Google.
    *   Informações importantes para a narrativa devem estar em negrito como o exemplo: <strong>Importante</strong>.
    *   CASO HAJAM OPÇÕES NO JOGO, elas devem ser claras e bem definidas, seguindo o padrão de formatação abaixo:
        <ol>
            <li>Opção 1 - Descrição</li>
            <li>Opção 2 - Descrição</li>
            <li>Opção 3 - Descrição</li>
            <li>Opção 4 - Descrição</li>
        </ol>

    '''
    )

prompt = ('''
    Inicie a história!
    Primeiro, apresente-se brevemente como o narrador, incorporando a persona que foi criada para você.
    Em seguida, descreva em dois parágrafos o mundo ou o cenário inicial da história, usando os detalhes que foram configurados.
    Finalmente, introduza o personagem do jogador, descreva onde ele se encontra e apresente a situação inicial que dá o pontapé na aventura. Termine com uma pergunta ou opções para o jogador tomar a primeira decisão.
    ''')

chat = client.chats.create(model=modelo_grande, config=chat_config) # Usando modelo_grande para o chat principal para melhor narrativa

# Loop principal do jogo
while prompt != "!FIM":
  try:
    resposta = chat.send_message(prompt)
    # Usar a função formatada para exibir a resposta do narrador
    display_narrator_response(resposta.text)

  except Exception as e:
      # Tratar possíveis erros da API
      display(Markdown(f"<div style='color: red;'>🚫 **Erro:** {e}</div>"))
      display(Markdown("Ocorreu um erro na comunicação com o Narrador. Tente digitar novamente ou digite `!FIM` para encerrar."))

  # Exibir o prompt para o usuário antes de pedir a entrada
  display(Markdown("\n<div style='font-weight: bold;'>✍️ SUA VEZ:</div>"))

  # Coletar a entrada do usuário
  prompt = input("") # O input em si não pode ser estilizado facilmente, mas o texto acima sim.

# Mensagem de fim de jogo
display_section_heading("Fim da Aventura")
display(Markdown("\n**Obrigado por jogar!** A história terminou (por enquanto!).\n"))



